THIS CODE HAS NOT BEEN RAN/TESTED, PLEASE DO SO
We replace the lines:

  # Cross-validation scores
    scores = cross_val_score(model, X, y, cv=5, scoring="r2")
    print(f"{model_name} Cross-Validation R²: {scores.mean():.3f} ± {scores.std():.3f}")

of [4] of 4. Training of Predictors.ipynb by the following:

1. Create a New Function for Walk-Forward Validation:

In [ ]:
def perform_walk_forward_validation(
    model_name: str,
    model: Any,
    X_df: pd.DataFrame,
    y: pd.Series,
    test_size: float = 0.1,
    seed: int = 0,
    custom_performance_metric=None,
):
    """
    Performs walk-forward validation on a given model and returns performance metrics.

    Args:
        model_name (str): Name of the model.
        model (Any): The machine learning model to be evaluated.
        X_df (pd.DataFrame): The feature data.
        y (pd.Series): The target variable (e.g., portfolio returns).
        test_size (float, optional): Proportion of data for testing in each step. Defaults to 0.1.
        seed (int, optional): Random seed for reproducibility. Defaults to 0.
        custom_performance_metric (function, optional): A user-defined function for calculating a custom metric.

    Returns:
        dict: A dictionary containing performance metrics (e.g., errors, custom metrics).
    """

    OUTPUT_DIR = "results"  # Adjust directory path as needed

    # Data preparation (can be moved to a separate function if needed)
    X = prepare_data(X_df)  # Assuming you have a prepare_data function for normalization/PCA

    # Walk-forward validation loop
    predictions = []
    errors = []
    actual_values = []
    for i in range(test_size, len(y)):
        train_X = X[:i]
        train_y = y[:i]
        test_X = X[i:i + 1]
        test_y = y[i:i + 1]

        model.fit(train_X, train_y)
        prediction = model.predict(test_X)[0]
        predictions.append(prediction)
        actual_values.append(test_y.values[0])
        error = mean_squared_error(test_y.values[0], prediction)
        errors.append(error)

    # Calculate additional performance metrics (if needed)
    if custom_performance_metric is not None:
        custom_metric_value = custom_performance_metric(actual_values, predictions)
    else:
        custom_metric_value = None

    # Store results in a dictionary
    results = {
        "model_name": model_name,
        "errors": errors,
        "custom_metric": custom_metric_value,  # Replace with actual metric name
    }

    # Save results (optional)
    # ... (code to save results to a file or database)

    return results


2. Call the Walk-Forward Function:

Keep your existing run_model_and_plot function for its intended purpose (traditional train-test split evaluation).
When you want to perform walk-forward validation, call the new perform_walk_forward_validation function after preparing your data:

In [ ]:
# Assuming you have prepared your data (X_df, y) and model (model)

# Use run_model_and_plot for traditional evaluation (optional)
# run_model_and_plot(model_name="MyModel", model=model, X_df=X_df, y=y)

# Perform walk-forward validation
validation_results = perform_walk_forward_validation(
    model_name="MyModel",
    model=model,
    X_df=X_df,
    y=y,
    custom_performance_metric=custom_metric,  # Optional custom metric
)

# Access and analyze the results
print(f"Walk-Forward Validation Results for {validation_results['model_name']}")
print(f"Errors: {validation_results['errors']}")
print(f"Custom Metric: {validation_results['custom_metric']}")  # Replace with actual metric name
